# Transfer Learning with ResNet50 and CIFAR-10 dataset
<hr>

## 1. Prepare & Explore Dataset

In [1]:
# import pachages 
from tensorflow import keras 
from keras.datasets import cifar10
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical 

from keras.applications import ResNet50

from keras.optimizers import SGD,Adam

In [2]:
# load data.
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.3)

In [4]:
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

((35000, 32, 32, 3), (35000, 1))
((15000, 32, 32, 3), (15000, 1))
((10000, 32, 32, 3), (10000, 1))


In [5]:
# We have 10 classes, so, our network will have 10 output neurons
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test) 

In [6]:
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

((35000, 32, 32, 3), (35000, 10))
((15000, 32, 32, 3), (15000, 10))
((10000, 32, 32, 3), (10000, 10))


We can now begin the actual process of model building.
The following a set process and following consistently makes learning this easier :

- Define the Data Augmentation (ImageDataGenerator) 
- Build the model (Base Model + Flatten + Dense)
- Check model summary
- Initialize Batch Size,Number of Epochs
- Compile model
- Fit the model 
- Evaluate the model 

In [7]:
#Data Augmentation Function: Let's define an instance of the ImageDataGenerator class and set the parameters.
#We have to instantiate for the Train,Validation and Test datasets
train_generator = ImageDataGenerator(
                                    rotation_range=2, 
                                    horizontal_flip=True,
                                    zoom_range=.1 )

val_generator = ImageDataGenerator(
                                    rotation_range=2, 
                                    horizontal_flip=True,
                                    zoom_range=.1)

train_generator.fit(x_train)
val_generator.fit(x_val)

## 2. Define the neural network architecture

In [10]:
# define the CNN model
'For the 2nd base model we will use Resnet 50 and compare the performance against the previous one'
'The hypothesis is that Resnet 50 should perform better because of its deeper architecture'
base_model_2 = ResNet50(include_top=False, weights='imagenet', input_shape=(32,32,3), classes=y_train.shape[1])

94773248/94765736 [==============================] - 14s 0us/step


In [11]:
#Lets add the final layers to these base models where the actual classification is done in the dense layers
#Since we have already defined Resnet50 as base_model_2, let us build the sequential model.
model_2 = Sequential()
#Add the Dense layers along with activation and batch normalization
model_2.add(base_model_2)
model_2.add(Flatten())

#Add the Dense layers along with activation and batch normalization
model_2.add(Dense(1024,activation=('relu'),input_dim=512))
model_2.add(Dense(512,activation=('relu'))) 
model_2.add(Dense(256,activation=('relu'))) 
model_2.add(Dropout(.3))#Adding a dropout layer that will randomly drop 30% of the weights
model_2.add(Dense(128,activation=('relu')))
model_2.add(Dropout(.2))
model_2.add(Dense(10,activation=('softmax'))) #This is the classification layer

'''model_2.add(Dense(4000,activation=('relu'),input_dim=512))
model_2.add(Dense(2000,activation=('relu'))) 
model_2.add(Dropout(.4))
model_2.add(Dense(1000,activation=('relu'))) 
model_2.add(Dropout(.3))#Adding a dropout layer that will randomly drop 30% of the weights
model_2.add(Dense(500,activation=('relu')))
model_2.add(Dropout(.2))
model_2.add(Dense(10,activation=('softmax'))) #This is the classification layer'''

In [12]:
print(model_2.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 4000)              8196000   
_________________________________________________________________
dense_1 (Dense)              (None, 2000)              8002000   
_________________________________________________________________
dropout (Dropout)            (None, 2000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              2001000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0

## 3. Compile the neural net

In [13]:
# compile your model
model_2.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

## 4. Fit / train the neural net

In [14]:
model_2.fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_val, y_val), verbose=1) 

Epoch 1/20
350/350 [==============================] - 990s 3s/step - loss: 2.1358 - accuracy: 0.2356 - val_loss: 1.3044 - val_accuracy: 0.5507
Epoch 2/20
350/350 [==============================] - 968s 3s/step - loss: 1.1031 - accuracy: 0.6161 - val_loss: 0.9298 - val_accuracy: 0.6858
Epoch 3/20
350/350 [==============================] - 968s 3s/step - loss: 0.7916 - accuracy: 0.7323 - val_loss: 0.8582 - val_accuracy: 0.7109
Epoch 4/20
350/350 [==============================] - 988s 3s/step - loss: 0.6036 - accuracy: 0.7965 - val_loss: 0.7845 - val_accuracy: 0.7419
Epoch 5/20
350/350 [==============================] - 1011s 3s/step - loss: 0.4691 - accuracy: 0.8404 - val_loss: 0.8299 - val_accuracy: 0.7398
Epoch 6/20
226/350 [==================>...........] - ETA: 5:59 - loss: 0.3594 - accuracy: 0.8808

KeyboardInterrupt: 

## 5. Evaluate the neural net

In [15]:
score = model_2.evaluate(x_val, y_val, verbose=1) 
print('Test accuracy:', score[1]) 

469/469 [==============================] - 33s 68ms/step - loss: 0.9280 - accuracy: 0.7317
Test accuracy: 0.7317333221435547


## 6. Make predictions / classifications for unseen data

In [ ]:
#not yet until we enhanced the results
predictions = model_2.predict(x_test)
predictions

<hr>